In [1]:
%cd /work/adapting-CLIP-VGPs/

/work/adapting-CLIP-VGPs


In [2]:
def Jaccard_sim(x, y):
    phrase_set1 = set(x.split('+'))
    phrase_set2 = set(y.split('+'))

    intersect = len(phrase_set1.intersection(phrase_set2))
    union = len(phrase_set1.union(phrase_set2))
    return intersect/union

Jaccard_sim('blue+hard+hat','hard+hat')

0.6666666666666666

In [3]:
from tqdm import tqdm
import csv

val_path = '/work/adapting-CLIP-VGPs/data/flickr/phrases_data/phrase_pair_remove_trivial_match_val.csv'

phrases = []
scores = []
gt = []

with open(val_path, encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for i, row in tqdm(enumerate(reader)):
        phrase1 = row['phrase1']
        phrase2 = row['phrase2']
        label = row['ytrue']
        scores.append(Jaccard_sim(phrase1, phrase2))
        phrases.append([phrase1, phrase2])
        gt.append(row['ytrue']=='True')
        

0it [00:00, ?it/s]

80813it [00:00, 85492.60it/s]


In [4]:
import numpy as np

thres = 0
np_scores = np.array(scores)
preds = np_scores>thres

In [5]:
from sklearn.metrics import precision_score, recall_score, f1_score

f1 = f1_score(gt, preds)
prec = precision_score(gt, preds)
rec = recall_score(gt, preds)

print(f'F1 Score: {f1}')
print(f'Precision: {prec}')
print(f'Recall: {rec}')

F1 Score: 0.6170108429139649
Precision: 0.7469656992084432
Recall: 0.5255731922398589


In [6]:
def find_best_f1(scores, gt):
    step = 0.001
    cur_best_f1 = 0
    cur_prec = 0
    cur_rec = 0
    cur_best_thres = 0
    for thres in np.arange(cur_best_thres, 1, step):
        new_pred = scores > thres
        f1 = f1_score(gt, new_pred)

        if f1 > cur_best_f1:
            cur_best_thres = thres
            cur_best_f1 = f1
            cur_prec = precision_score(gt, new_pred)
            cur_rec = recall_score(gt, new_pred)
            
    return cur_best_thres, cur_best_f1, cur_prec, cur_rec

In [7]:
find_best_f1(np_scores, gt)

(0.0, 0.6170108429139649, 0.7469656992084432, 0.5255731922398589)

In [8]:
thres = 0

test_path = '/work/adapting-CLIP-VGPs/data/flickr/phrases_data/phrase_pair_remove_trivial_match_test.csv'

# phrases = []
# scores = []
preds = []
gt = []

with open(test_path, encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for i, row in tqdm(enumerate(reader)):
        phrase1 = row['phrase1']
        phrase2 = row['phrase2']
        # label = row['ytrue']
        score = Jaccard_sim(phrase1, phrase2)
        preds.append(score>thres)
        # phrases.append([phrase1, phrase2])
        gt.append(row['ytrue']=='True')
        

37155it [00:00, 93646.00it/s]

81285it [00:00, 93533.83it/s]


In [9]:
f1 = f1_score(gt, preds)
prec = precision_score(gt, preds)
rec = recall_score(gt, preds)

print(f'F1 Score: {f1}')
print(f'Precision: {prec}')
print(f'Recall: {rec}')

F1 Score: 0.6124667137655562
Precision: 0.7413498223917906
Recall: 0.5217592592592593
